In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col

In [2]:
spark = SparkSession.builder.appName('finalproject')\
        .config('spark.driver.extraClassPath','/usr/lib/jvm/java-17-openjdk-amd64/lib/postgresql-42.6.0.jar')\
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/12 00:01:37 WARN Utils: Your hostname, SUSHAN resolves to a loopback address: 127.0.1.1; using 172.22.228.148 instead (on interface eth0)
23/09/12 00:01:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 00:01:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
stationInfo_df = spark.read.csv("data/Fuel_Station_Information.csv", header=True, inferSchema=True)
hourlyPrices_df = spark.read.csv("data/Hourly_Gasoline_Prices.csv", header=True, inferSchema=True)

In [4]:
print(hourlyPrices_df.printSchema(), stationInfo_df.printSchema())

root
 |-- Id: integer (nullable = true)
 |-- isSelf: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- Date: timestamp (nullable = true)

root
 |-- Id: string (nullable = true)
 |-- Fuel_station_manager: string (nullable = true)
 |-- Petrol_company: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Station_name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitudine: string (nullable = true)

None None


In [5]:
# Remove Duplicates
hourlyPrices_df = hourlyPrices_df.dropDuplicates()
stationInfo_df = stationInfo_df.dropDuplicates()

# Drop rows with any null values
hourlyPrices_df = hourlyPrices_df.dropna()
stationInfo_df = stationInfo_df.dropna()


In [6]:
hourlyPrices_df = hourlyPrices_df.withColumn("Date", to_timestamp(col("Date"), "yyyy-MM-dd HH:mm:ss"))

In [7]:
countt = hourlyPrices_df.count()
print("Total number of records in hourlyPrices:", countt)
counttt = stationInfo_df.count()
print("Total number of records in stationInfo:", counttt)

Total number of records in hourlyPrices: 2444337
Total number of records in stationInfo: 22129


In [8]:
hourlyPrices_df.write.parquet("data/cleaned_fuel_prices.parquet", mode= "overwrite", compression= "snappy")
stationInfo_df.write.parquet("data/cleaned_station_info.parquet", mode= "overwrite", compression= "snappy")

23/09/12 00:05:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [20]:
# Read the Parquet files
hourlyPrices_df = spark.read.parquet("data/cleaned_fuel_prices.parquet")
stationInfo_df = spark.read.parquet("data/cleaned_station_info.parquet")

In [21]:
from pyspark.sql.types import FloatType

stationInfo_df = stationInfo_df.withColumn("Latitude", stationInfo_df["Latitude"].cast(FloatType()))
stationInfo_df = stationInfo_df.withColumn("Longitudine", stationInfo_df["Longitudine"].cast(FloatType()))

In [22]:
print(hourlyPrices_df.printSchema(), stationInfo_df.printSchema())

root
 |-- Id: integer (nullable = true)
 |-- isSelf: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- Date: timestamp (nullable = true)

root
 |-- Id: string (nullable = true)
 |-- Fuel_station_manager: string (nullable = true)
 |-- Petrol_company: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Station_name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitudine: float (nullable = true)

None None


In [23]:

# Define the JDBC connection properties
jdbc_url = "jdbc:postgresql://localhost:5432/sparkProject"
jdbc_properties = {
    "user": "sushan",
    "password": "7446",
    "driver": "org.postgresql.Driver"
}

In [24]:

# Save DataFrames to PostgreSQL tables
hourlyPrices_df.write.jdbc(url=jdbc_url, table="hourly_gasoline_prices", mode="overwrite", properties=jdbc_properties)
stationInfo_df.write.jdbc(url=jdbc_url, table="fuel_station_information", mode="overwrite", properties=jdbc_properties)


In [ ]:
# Stop the SparkSession
spark.stop()